# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

data_df = pd.read_csv('../WeatherPy/city_weather_data.csv')
# Drop unnamed column/index artifact.
data_df = data_df.drop(data_df.columns[0], axis=1)
data_df.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kumul,42.8000,93.4500,48.13,35,1,4.88,CN,1615674990
1,Boralday,43.3578,76.8594,21.20,93,90,8.95,KZ,1615675223
2,Longyearbyen,78.2186,15.6401,-5.21,94,95,4.61,SJ,1615675068
3,Rikitea,-23.1203,-134.9692,80.47,73,39,8.34,PF,1615675224
4,Conceição do Araguaia,-8.2578,-49.2647,75.61,88,57,1.57,BR,1615675225


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# See https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html for usage
locations = data_df[['Lat', 'Lon']]
weights = data_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,  dissipating=False))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df = data_df[
                        (data_df['Max Temp'] < 80) 
                        & (data_df['Max Temp'] > 70)
                        & (data_df['Wind Speed'] < 10)
                        & (data_df['Cloudiness'] == 0)
                    ]
print(len(ideal_weather_df))
ideal_weather_df.head()

19


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,Coyhaique,-45.5752,-72.0662,71.60,37,0,6.91,CL,1615675042
72,Saint-Philippe,-21.3585,55.7679,75.20,73,0,5.75,RE,1615675052
101,Ballitoville,-29.5390,31.2144,75.99,88,0,6.91,ZA,1615675273
104,São Filipe,14.8961,-24.4956,70.79,76,0,5.39,CV,1615675274
171,Port Elizabeth,-33.9180,25.5701,73.40,53,0,5.75,ZA,1615675311


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame()

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'radius': 5000,
    'types': 'hotel',
    'key': g_key
}

for index, row in ideal_weather_df.iterrows():
    params['location'] = f"{row['Lat']},{row['Lon']}"
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        # Only care about the first result
        hotel_data = hotel_data['results'][0]
        tmp_df = pd.DataFrame([{
                        'Hotel Name': hotel_data['name'],
                        'Lat': hotel_data['geometry']['location']['lat'],
                        'Lng': hotel_data['geometry']['location']['lng'],
                        'City': row['City'],
                        'Country': row['Country']
                    }])
        hotel_df = hotel_df.append(tmp_df, ignore_index=True)
    # In case there is no result returned or fields are missing
    except (KeyError, IndexError):
        print(f"One or more data points missing")

hotel_df

One or more data points missing


,Hotel Name,Lat,Lng,City,Country
0,Coyhaique,-45.571225,-72.068265,Coyhaique,CL
1,Saint-Philippe,-21.358118,55.764656,Saint-Philippe,RE
2,Dolphin Coast,-29.437578,31.222571,Ballitoville,ZA
3,Sao Filipe,14.895168,-24.494564,São Filipe,CV
4,Gqeberha,-33.960837,25.602242,Port Elizabeth,ZA
5,Ilo District,-17.668084,-71.346809,Ilo,PE
6,Sittwe,20.152766,92.867686,Sittwe,MM
7,Necochea,-38.554497,-58.739609,Necochea,AR
8,Morón,-34.655861,-58.616721,Morón,AR
9,Kanikeli,-12.955198,45.106200,Chirongui,YT


In [7]:
locations = hotel_df[['Lat', 'Lng']]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, dissipating=False))
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map


# Display figure
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))
fig

Figure(layout=FigureLayout(height='420px'))